In [23]:
# =============================================================================
# 🚀 Netflix Revenue Optimization Model - Robust Deployment (Unique Endpoint)
# =============================================================================
import os
import tarfile
import boto3
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from datetime import datetime
import botocore.exceptions
import time

# =============================================================================
# CONFIGURATION
# =============================================================================
REGION = "us-east-1"
BUCKET_NAME = "netflix-revenue-opt-2025-keith-xyz123"
MODEL_FILE = "revenue_optimization_model.pkl"
MODEL_NAME_BASE = "netflix-revenue-model"
ROLE_ARN = "arn:aws:iam::619071352851:role/SageMaker-Execution-Role"

# Generate unique timestamp
TIMESTAMP = datetime.now().strftime("%Y%m%d-%H%M%S")
ENDPOINT_NAME = f"netflix-revenue-endpoint-{TIMESTAMP}"  # Unique endpoint

# =============================================================================
# 1️⃣ TRAIN & SAVE MODEL
# =============================================================================
def train_model():
    print("📊 Training revenue optimization model...")

    np.random.seed(42)
    data = pd.DataFrame({
        "subscribers": np.random.randint(100_000, 10_000_000, 100),
        "avg_watch_hours": np.random.uniform(1, 10, 100),
        "monthly_fee": np.random.uniform(5, 20, 100),
        "marketing_spend": np.random.uniform(10_000, 500_000, 100),
    })
    data["revenue"] = data["subscribers"] * data["monthly_fee"] * np.random.uniform(0.95, 1.05, 100)

    X = data[["subscribers", "avg_watch_hours", "monthly_fee", "marketing_spend"]]
    y = data["revenue"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    r2 = r2_score(y_test, preds)
    print(f"✅ Model trained successfully with R² score: {r2:.4f}")

    os.makedirs("model", exist_ok=True)
    local_model_path = os.path.join("model", MODEL_FILE)
    joblib.dump(model, local_model_path)
    print(f"✅ Model saved locally at: {local_model_path}")
    
    return local_model_path

# =============================================================================
# 2️⃣ CREATE INFERENCE SCRIPT
# =============================================================================
def create_inference_script():
    inference_code = '''
import json
import joblib
import pandas as pd
import os

MODEL_FILE = 'revenue_optimization_model.pkl'

def model_fn(model_dir):
    return joblib.load(os.path.join(model_dir, MODEL_FILE))

def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        df = pd.DataFrame(json.loads(request_body))
        return df[["subscribers", "avg_watch_hours", "monthly_fee", "marketing_spend"]]
    raise ValueError(f"Unsupported content type: {request_content_type}")

def predict_fn(input_data, model):
    return model.predict(input_data)

def output_fn(prediction, content_type):
    if content_type == 'application/json':
        return json.dumps(prediction.tolist())
    return str(prediction)
'''
    with open('inference.py', 'w') as f:
        f.write(inference_code)
    print("✅ Created inference.py script")

# =============================================================================
# 3️⃣ PACKAGE MODEL
# =============================================================================
def package_model(model_path):
    tar_file = f"{MODEL_NAME_BASE}-{TIMESTAMP}.tar.gz"
    with tarfile.open(tar_file, "w:gz") as tar:
        tar.add(model_path, arcname=MODEL_FILE)
        tar.add('inference.py', arcname='inference.py')
    print(f"✅ Model package created: {tar_file}")
    return tar_file

# =============================================================================
# 4️⃣ UPLOAD TO S3
# =============================================================================
def upload_to_s3(tar_file, bucket, region):
    print(f"\n☁️ Uploading {tar_file} to S3 bucket {bucket}...")
    s3 = boto3.client("s3", region_name=region)
    try:
        if region == "us-east-1":
            s3.create_bucket(Bucket=bucket)
        else:
            s3.create_bucket(
                Bucket=bucket,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"✅ Bucket '{bucket}' created or already exists.")
    except (s3.exceptions.BucketAlreadyOwnedByYou, s3.exceptions.BucketAlreadyExists):
        print(f"ℹ️ Bucket '{bucket}' already exists.")
    s3.upload_file(tar_file, bucket, tar_file)
    model_s3_uri = f"s3://{bucket}/{tar_file}"
    print(f"✅ Model uploaded to {model_s3_uri}")
    return model_s3_uri

# =============================================================================
# 5️⃣ DEPLOY TO SAGEMAKER
# =============================================================================
def deploy_model(model_s3_uri, role, region):
    print("\n⚙️ Deploying model to SageMaker...")
    sm = boto3.client("sagemaker", region_name=region)

    model_name = f"{MODEL_NAME_BASE}-{TIMESTAMP}"
    endpoint_config_name = f"{ENDPOINT_NAME}-config"

    # Create Model
    sm.create_model(
        ModelName=model_name,
        PrimaryContainer={
            "Image": f"683313688378.dkr.ecr.{region}.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3",
            "ModelDataUrl": model_s3_uri,
            "Environment": {
                'SAGEMAKER_PROGRAM': 'inference.py',
                'SAGEMAKER_SUBMIT_DIRECTORY': model_s3_uri
            }
        },
        ExecutionRoleArn=role,
    )
    print(f"✅ SageMaker model created: {model_name}")

    # Create Endpoint Config
    sm.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": "ml.m5.large",
        }]
    )
    print(f"✅ Endpoint configuration created: {endpoint_config_name}")

    # Create Endpoint
    sm.create_endpoint(
        EndpointName=ENDPOINT_NAME,
        EndpointConfigName=endpoint_config_name
    )
    print(f"🚀 Endpoint creation initiated: {ENDPOINT_NAME}")

    # Wait until endpoint is InService
    print(f"⏳ Waiting for endpoint '{ENDPOINT_NAME}' to be InService (up to 30 mins)...")
    waiter = sm.get_waiter('endpoint_in_service')
    try:
        waiter.wait(EndpointName=ENDPOINT_NAME, WaiterConfig={'Delay': 30, 'MaxAttempts': 60})
        print("✅ Endpoint is now InService!")
    except botocore.exceptions.WaiterError as e:
        print(f"❌ Endpoint failed to reach InService: {e}")

    return model_name, ENDPOINT_NAME

# =============================================================================
# 6️⃣ CLEANUP FUNCTION (Optional)
# =============================================================================
def delete_endpoint(model_name, endpoint_name, region):
    sm = boto3.client("sagemaker", region_name=region)
    try:
        sm.delete_endpoint(EndpointName=endpoint_name)
        sm.delete_endpoint_config(EndpointConfigName=f"{endpoint_name}-config")
        sm.delete_model(ModelName=model_name)
        print(f"✅ Successfully deleted endpoint, config, and model: {endpoint_name}")
    except Exception as e:
        print(f"❌ Cleanup failed: {e}")

# =============================================================================
# 7️⃣ MAIN PIPELINE
# =============================================================================
def main():
    print("🚀 Starting Netflix Revenue Optimization Deployment")

    create_inference_script()
    model_path = train_model()
    tar_file = package_model(model_path)
    model_s3_uri = upload_to_s3(tar_file, BUCKET_NAME, REGION)
    model_name, endpoint_name = deploy_model(model_s3_uri, ROLE_ARN, REGION)

    print("\n🎉 DEPLOYMENT COMPLETE!")
    print(f"📦 Model: {model_name}")
    print(f"🌐 Endpoint: {endpoint_name}")
    print(f"🔗 Endpoint URL: https://runtime.sagemaker.{REGION}.amazonaws.com/endpoints/{endpoint_name}/invocations")

    # OPTIONAL: Cleanup to stop charges
    # delete_endpoint(model_name, endpoint_name, REGION)

if __name__ == "__main__":
    main()


🚀 Starting Netflix Revenue Optimization Deployment
✅ Created inference.py script
📊 Training revenue optimization model...
✅ Model trained successfully with R² score: 0.9217
✅ Model saved locally at: model\revenue_optimization_model.pkl
✅ Model package created: netflix-revenue-model-20251025-134503.tar.gz

☁️ Uploading netflix-revenue-model-20251025-134503.tar.gz to S3 bucket netflix-revenue-opt-2025-keith-xyz123...
✅ Bucket 'netflix-revenue-opt-2025-keith-xyz123' created or already exists.
✅ Model uploaded to s3://netflix-revenue-opt-2025-keith-xyz123/netflix-revenue-model-20251025-134503.tar.gz

⚙️ Deploying model to SageMaker...
✅ SageMaker model created: netflix-revenue-model-20251025-134503
✅ Endpoint configuration created: netflix-revenue-endpoint-20251025-134503-config
🚀 Endpoint creation initiated: netflix-revenue-endpoint-20251025-134503
⏳ Waiting for endpoint 'netflix-revenue-endpoint-20251025-134503' to be InService (up to 30 mins)...
✅ Endpoint is now InService!

🎉 DEPLOYMEN

In [24]:
import boto3
import json

# Replace with your unique endpoint name
ENDPOINT_NAME = "netflix-revenue-endpoint-20251025-134503"
REGION = "us-east-1"

# Sample input data
payload = [
    {
        "subscribers": 2_000_000,
        "avg_watch_hours": 5.5,
        "monthly_fee": 12.99,
        "marketing_spend": 200_000
    },
    {
        "subscribers": 5_000_000,
        "avg_watch_hours": 7.2,
        "monthly_fee": 14.99,
        "marketing_spend": 350_000
    }
]

# Initialize SageMaker runtime client
runtime = boto3.client("sagemaker-runtime", region_name=REGION)

# Invoke endpoint
response = runtime.invoke_endpoint(
    EndpointName=ENDPOINT_NAME,
    ContentType="application/json",
    Body=json.dumps(payload)
)

# Parse response
predictions = json.loads(response['Body'].read().decode())
print("Predicted revenue:", predictions)


Predicted revenue: [26251596.629171647, 75322024.19901049]
